In [87]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [88]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [89]:
df = pd.read_csv("urls.csv")
urls = df["Address"].tolist()
url_sent_score = []
url_sent_label = []
total_pos = []
total_neg = []

In [90]:
for count, x in enumerate(urls):
  url = x
  
  headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
  res = requests.get(url,headers=headers)
  html_page = res.text

  soup = BeautifulSoup(html_page, 'html.parser')
  for script in soup(["script", "style","meta","label","header","footer"]):
    script.decompose()
  page_text = (soup.get_text()).lower()
  page_text = page_text.strip().replace("  ","")
  page_text = "".join([s for s in page_text.splitlines(True) if s.strip("\r\n")])

In [91]:
doc = nlp(page_text)
sentiment = doc._.blob.polarity
sentiment = round(sentiment,2)

if sentiment > 0:
  sent_label = "Positive"
else:
  sent_label = "Negative"

url_sent_label.append(sent_label)
url_sent_score.append(sentiment)

In [92]:
positive_words = []
negative_words = []

for x in doc._.blob.sentiment_assessments.assessments:
  if x[1] > 0:
    positive_words.append(x[0][0])
  elif x[1] < 0:
    negative_words.append(x[0][0])
  else:
    pass

total_pos.append(', '.join(set(positive_words)))
total_neg.append(', '.join(set(negative_words)))

In [93]:
df["Sentiment Score"] = pd.Series(url_sent_score)
df["Sentiment Label"] = pd.Series(url_sent_label)
df["Positive Words"] = pd.Series(total_pos)
df["Negative Words"] = pd.Series(total_neg)

df

,Address,Sentiment Score,Sentiment Label,Positive Words,Negative Words
0,https://www.google.com/?client=safari,-0.22,Negative,,"center, disabled, not, wrong"
1,https://twitter.com/i/events/1574545709998649344,NaN,NaN,NaN,NaN
2,https://twitter.com/i/events/1575929010806063106,NaN,NaN,NaN,NaN
3,https://twitter.com/i/events/1575926927621226512,NaN,NaN,NaN,NaN
